In [27]:
import numpy as np
import collections

class BTreeNode:
    def __init__(self, t, leaf):
        self.keys = []
        self.children = []
        self.leaf = leaf
        self.t = t

class BTree:
    def __init__(self, t):
        self.root = None
        self.t = t
        self.disk_accesses = 0
        self.cache = collections.OrderedDict()
        self.cache_capacity = 64 * 1024 // (t * 2 + 1)

    def search(self, k):
        if self.root is None:
            return None
        return self._search(self.root, k)

    def _search(self, node, k):
        i = 0
        while i < len(node.keys) and k > node.keys[i][0]:
            i += 1
        if i < len(node.keys) and k == node.keys[i][0]:
            return node
        elif node.leaf:
            return None
        else:
            if node.children[i] in self.cache:
                child = self.cache[node.children[i]]
            else:
                child = self._read_node(node.children[i])
                self.cache[node.children[i]] = child
            self.disk_accesses += 1
            return self._search(child, k)

    def insert(self, k):
        if self.root is None:
            self.root = BTreeNode(self.t, True)
            self.root.keys.append(k)
        else:
            if len(self.root.keys) == 2 * self.t - 1:
                new_root = BTreeNode(self.t, False)
                new_root.children.append(self.root)
                self._split_child(new_root, 0, self.root)
                self.root = new_root
            self._insert(self.root, k)

    def _insert(self, node, k):
        i = len(node.keys) - 1
        print(node.keys)
        print(k)
        if node.leaf:
            while i >= 0 and k[0] < node.keys[i][0]:
                node.keys[i + 1] = node.keys[i][0]
                i -= 1
            node.keys[i + 1] = k
        else:
            while i >= 0 and k[0] < node.keys[i][0]:
                i -= 1
            if len(node.children[i + 1].keys) == 2 * self.t - 1:
                self._split_child(node, i + 1, node.children[i + 1])
                if k[0] > node.keys[i + 1]:
                    i += 1
            self._insert(node.children[i + 1], k)

    def _split_child(self, node, i, y):
        z = BTreeNode(y.t, y.leaf)
        node.children.insert(i + 1, z)
        node.keys.insert(i, y.keys[y.t - 1])
        z.keys = y.keys[y.t : 2 * y.t - 1]
        y.keys = y.keys[0 : y.t - 1]
        if not y.leaf:
            z.children = y.children[y.t : 2 * y.t]
            y.children = y.children[0 : y.t]

    def _read_node(self, node_id):
        # Here, we simulate reading a node from disk by generating a new node
        # with randomly generated keys and children.
        node = BTreeNode(self.t, False)
        node.keys = [k + 1 for k in np.random.permutation(1000)[:2 * self.t - 1]]
        node.children = [i for i in range(len(node.keys) + 1)]
        return node    

In [28]:
key_list = [k + 1 for k in np.random.permutation(100000)]
search_list = [k + 1 for k in np.random.permutation(100000)]

In [29]:
B = BTree(3)

for i in range(10):
    print(10 + 2 *i)
    B.insert((i, 10 + 2 *i))

10
12
[(0, 10)]
(1, 12)


IndexError: list assignment index out of range